## DL Assignment 2

### Team-  Apurva Shekhar || Dnyanai Surkutwar || Ritu Ranjani Ravi Shankar || Suchita Negi

### 1. Consider the following computational graph:<img src='imgs/hw2-Q1.jpg'><br>Use the Gradient Flow Rules to compute the gradients for all the arcs in the graph.<br><br>(2 points)

<b>Answer:</b>

### 2. Consider a Deep Feed Forward Network, of the type shown on Page 16 of Lecture 4 Slides,composed of an Input Layer with 10 nodes, followed by a Hidden Layer with 50 nodes and finally an Output Layer with 3 nodes. Assume the activation function f is given by ReLU, and the output function h is given by Softmax.<br><ul><li>a.  Compute the number of parameters (weights and biases) required to describe the network.</li><br><li>b. Write a Python program to do a forward pass through the network. Assume that the input training sample X = (0.5, 0.6, 0.1, 0.25, 0.33, 0.9, 0.88, 0.76, 0.69, 0.95). Also assume that all the weights and biases are initialized according to a Uniform distribution between [0,0.1]</li><br><li>c. Assuming that the output Label corresponding to X, is T = (1, 0, 0). Using the results of part (b), compute the Cross Entropy Loss Function.In order to verify the forward pass, the Loss Function should compute to log(3),explain why.</li><br><li>d. Write a Python program to do a backward pass through the network, and compute all the delta values.</li><br><li>e. Finally extend the Python program to compute the new weight parameters using stochastic gradient descent.<br>In order to verify that the algorithm has been coded correctly, you can do the following: Put the forward and backward passes in a loop (with the single training sample given in Part (b) and the corresponding Label in Part (c)). After several iterations you should see the Loss Function go to zero (explain why).</ul><br><br>(15 points)

<b>Answer:</b>

## a) <img src="imgs/hw2-Q2a.jpg">

## b) Feedforward Neural Network - 

In [4]:
import numpy as np

class model_FFNN:
    
    def __init__(self,X,T):
        self.X = X
        self.T = T ## T labels
        
    def model_layer_sizes(self):
        n_x = self.X.shape[0] ## 10 rows, 1 column 
        n_h = 50    
        n_y = self.T.shape[0] ## 3
        return (n_x,n_h,n_y)

    def parameters_initialize(self):
        
        np.random.seed(5)
        ## Wts and biases for the first layer i.e. hidden layer,
        W1 = np.random.uniform(0,0.1,(n_h,n_x))
        b1 = np.full((n_h,1),np.random.uniform(0,0.1))

        ## Wts and biases for the output layer,
        W2 = np.random.uniform(0,0.1,(n_y,n_h))
        b2 = np.full((n_y,1),(np.random.uniform(0,0.1)))

        assert (W1.shape == (n_h, n_x))
        assert (b1.shape == (n_h, 1))
        assert (W2.shape == (n_y, n_h))
        assert (b2.shape == (n_y, 1))
       
        params = {"W1":W1,"b1":b1,"W2":W2,"b2":b2}
        
        return params
    
    def pre_activation(self,layer,w,b): ## Step 1
        return (np.dot(w,layer)+b)
        
    def activation_ReLU(self,z):       ## Step 2
        return np.maximum(0,z)
        
    def output_Softmax(self,z2):      ## Step 3
        numerator = np.exp(z2)
        denominator = sum(np.exp(z2))
        return numerator/denominator
    
    ## To do:
    def backward_pass(self,):
        
        return 0
    

##Main function:
X_train = model_FFNN(np.array([[0.5],[0.6],[0.1],[0.25],[0.33],[0.9],[0.88],[0.76],[0.69],[0.95]]),np.array([[1],[0],[0]]))

    
## Step1: Setting up the model layers and their vector shapes:
(n_x,n_h,n_y) = X_train.model_layer_sizes()

print('The size of the input layer is: ',n_x)
print('The size of the hidden layer is: ',n_h)
print('The size of the output layer is: ',n_y)
print('-'*100)

## Step2: Creating parameters in accordance to the training samples for the respective layers of the model 
params = X_train.parameters_initialize()

print('The size of the weights for the 1st layer i.e. hidden layer is:',params["W1"].shape)
print('The size of the bias for the 1st layer i.e. hidden layer is:',params["b1"].shape)
print('The size of the weights for the 2st layer i.e. output layer is:',params["W2"].shape)
print('The size of the bias for the 2st layer i.e. output layer is:',params["b2"].shape)
print('-'*100)

## Step3: Caculate the forward pass: A1,Z1
# Pre-activation:
Z1 = X_train.pre_activation(X_train.X,params["W1"],params["b1"])    ## 1st layer - [wT].[x] + [b1] 
print('The size of the pre-activation calculation is :',Z1.shape)
print('-'*100)

A1 = X_train.activation_ReLU(Z1)                        ## Activation fn - ReLU
print('The size of the activation matrix is :',A1.shape)
print('-'*100)

Z2 = X_train.pre_activation(A1,params["W2"],params["b2"])              ## 2nd layer - [wT].[x] + [b2]
print('The size of the 2nd layer pre-activation is :',Z2.shape)
print('-'*100)

Y = X_train.output_Softmax(Z2) ## A2                   ## Output fn - Softmax
print('The size of the output layer is :',Y.shape)
print('-'*100)

print('The output vector is :\n',Y)
print('-'*100)

The size of the input layer is:  10
The size of the hidden layer is:  50
The size of the output layer is:  3
----------------------------------------------------------------------------------------------------
The size of the weights for the 1st layer i.e. hidden layer is: (50, 10)
The size of the bias for the 1st layer i.e. hidden layer is: (50, 1)
The size of the weights for the 2st layer i.e. output layer is: (3, 50)
The size of the bias for the 2st layer i.e. output layer is: (3, 1)
----------------------------------------------------------------------------------------------------
The size of the pre-activation calculation is : (50, 1)
----------------------------------------------------------------------------------------------------
The size of the activation matrix is : (50, 1)
----------------------------------------------------------------------------------------------------
The size of the 2nd layer pre-activation is : (3, 1)
-------------------------------------------------

## c) Cross Entropy, $- \sum\limits_{i=0}^{k}  t^{(i)}\log(Y{(i)})$ :

In [18]:
## Above T is referring to the Ground Truth and Y is referring to the forward pass output vector - 
logterm = np.multiply(np.log(Y),X_train.T)   ## (3,1) . (3,1) so k = 3
cross_entropy = -sum(logterm)
print('The Cross Entropy Loss is:\n ',cross_entropy[0])
print('\nTo verify the forward pass we will take log(3) in base e as np operations have been done in base e:\n',np.log(3))
print('-'*100)

The Cross Entropy Loss is:
  1.0503865969524828

To verify the forward pass we will take log(3) in base e as np operations have been done in base e:
 1.0986122886681098
----------------------------------------------------------------------------------------------------


### 3. The MNIST_Fashion dataset comes with Keras, and can be loaded using the commands: fashion_mnist = keras.datasets.fashion_mnist(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data() This dataset corresponds to grayscale coded images of various clothing items. Just as in MNIST, there are 10 classes and each image is a 28*28 matrix (see https://www.tensorflow.org/tutorials/keras/basic_classification for more information about this dataset). Answer the following:<br><ul><li>a. What is the best classification accuracy that you can achieve with the Linear Model on this dataset?</li><li>b. Replace the Linear model with a Fully Connected Feed Forward Model with one or more Hidden Layers. What is the best classification accuracy that you can achieve with this model? In order to get better performance , you can vary the number of hidden layers and the number of nodes per hidden layer</li></ul><br><br>(8 Points)

<b>Answer:</b>

### 4. The figure above shows an example of Ensemble Processing (see Lecture 4). Start with the best model that you have after doing Problem 3, and replicate that model 3 times in a parallel configuration as shown in the figure (use the Keras Functional API for doing this). Take the final logit nodes from each branch and average it to create the final logit layer for the model (see https://keras.io/api/layers/merging_layers/average/ on how to do this). Train the model again on the fashion_mnist dataset, and check for an improvement in prediction accuracy (if any).<img src="imgs/hw2-Q4.jpg"><br><br>(5 Points)

<b>Answer:</b>